mtask处理代码
1. 得到最基础的mtask_relative，会有识别不出来的情况
通过运行改版后的train.py，并且要把lerobot_dataset里面
```
        # 训练的时候要用
        # if self.language_tip_mode:
        #     tip_filename = f"mtask_{self.language_tip_mode}.jsonl"
        #     tip_file = self.root / "meta" / tip_filename
        #     if not tip_file.exists():
        #         raise FileNotFoundError(f"训练需要的文件不存在: {tip_file}")
        #     self.modified_tasks = load_modified_tasks(tip_file)
        #     print("self.modified_tasks举例：",self.modified_tasks[1][0])
```
和factory里面
```
        # exclude_features += ["observation.images.side_depth", "observation.images.side_depth_is_pad"]
```


2. 然后补完原始的数值

In [5]:
# 补完数值
import json
import re
from pathlib import Path


def parse_coords(text, key):
    """从 task 字符串里解析 key 对应的坐标 (x, y, z)。"""
    pattern = rf"{key} at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)"
    match = re.search(pattern, text)
    if match:
        return tuple(map(float, match.groups()))
    return None


def replace_coords(text, key, coords):
    """替换或插入 key 的坐标。"""
    text = text.replace("sachet", "sponge")  # 统一替换
    pattern = rf"{key} at \([^)]*\)"
    if coords is None:
        return text
    repl = f"{key} at ({coords[0]:.3f}, {coords[1]:.3f}, {coords[2]:.3f})"
    if re.search(pattern, text):
        return re.sub(pattern, repl, text)
    else:
        if "|" in text:
            return text.strip() + ", " + repl
        else:
            return text.strip() + " | " + repl


def linear_interpolate(start, end, n):
    """在 start 和 end 之间插值 n 步 (不含两端)。"""
    if start is None or end is None:
        return [None] * (n - 1)
    result = []
    for i in range(1, n):
        frac = i / n
        interp = tuple(s + (e - s) * frac for s, e in zip(start, end))
        result.append(interp)
    return result


def process_file(input_file, output_file):
    lines = [json.loads(line) for line in Path(input_file).read_text().splitlines()]

    modified, unmodified = 0, 0

    # 按 episode 分组
    episodes = {}
    for entry in lines:
        episodes.setdefault(entry["episode_index"], []).append(entry)

    for ep, entries in episodes.items():
        entries.sort(key=lambda x: x["frame_index"])

        # 找缺失的区间
        for i, entry in enumerate(entries):
            task = entry["task"]
            if "?" in task:
                # 向前找已知
                j = i - 1
                while j >= 0 and "?" in entries[j]["task"]:
                    j -= 1
                # 向后找已知
                k = i + 1
                while k < len(entries) and "?" in entries[k]["task"]:
                    k += 1

                if j >= 0 and k < len(entries):
                    # 插值
                    g1 = parse_coords(entries[j]["task"], "gripper")
                    g2 = parse_coords(entries[k]["task"], "gripper")
                    s1 = parse_coords(entries[j]["task"], "sponge")
                    s2 = parse_coords(entries[k]["task"], "sponge")

                    g_interp = linear_interpolate(g1, g2, k - j)
                    s_interp = linear_interpolate(s1, s2, k - j)

                    for m in range(j + 1, k):
                        if "?" in entries[m]["task"]:
                            new_task = "Pick up a yellow sponge and place it inside the box"
                            new_task = replace_coords(new_task, "gripper", g_interp[m - j - 1])
                            new_task = replace_coords(new_task, "sponge", s_interp[m - j - 1])
                            entries[m]["task"] = new_task
                            modified += 1
                else:
                    unmodified += 1
            else:
                # 只做 sachet -> sponge 替换
                if "sachet" in task:
                    entries[i]["task"] = task.replace("sachet", "sponge")

    # 保存
    with open(output_file, "w") as f:
        for ep in sorted(episodes):
            for entry in sorted(episodes[ep], key=lambda x: x["frame_index"]):
                f.write(json.dumps(entry, ensure_ascii=False) + "\n")

    print(f"修改成功: {modified}, 未修改成功: {unmodified}")


if __name__ == "__main__":
    input_file = "/home/zhonglinyue/code/lerobot_diy/training_dataset/831/sponge_pickup/meta/modified_tasks_pure.jsonl"
    output_file = "full_modified_tasks.jsonl"
    process_file(input_file, output_file)



修改成功: 331, 未修改成功: 25


In [7]:
# 处理两端的
import json
import re
from pathlib import Path


def parse_coords(text, key):
    pattern = rf"{key} at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)"
    match = re.search(pattern, text)
    if match:
        return tuple(map(float, match.groups()))
    return None


def replace_coords(text, key, coords):
    text = text.replace("sachet", "sponge")  # 统一替换
    pattern = rf"{key} at \([^)]*\)"
    if coords is None:
        return text
    repl = f"{key} at ({coords[0]:.3f}, {coords[1]:.3f}, {coords[2]:.3f})"
    if re.search(pattern, text):
        return re.sub(pattern, repl, text)
    else:
        if "|" in text:
            return text.strip() + ", " + repl
        else:
            return text.strip() + " | " + repl


def linear_interpolate(start, end, n):
    if start is None or end is None:
        return [None] * (n - 1)
    result = []
    for i in range(1, n):
        frac = i / n
        interp = tuple(s + (e - s) * frac for s, e in zip(start, end))
        result.append(interp)
    return result


def process_file(input_file, output_file):
    lines = [json.loads(line) for line in Path(input_file).read_text().splitlines()]

    modified, unmodified = 0, 0

    episodes = {}
    for entry in lines:
        episodes.setdefault(entry["episode_index"], []).append(entry)

    for ep, entries in episodes.items():
        entries.sort(key=lambda x: x["frame_index"])

        # 先处理开头缺失
        first_valid = next((e for e in entries if "?" not in e["task"]), None)
        if first_valid:
            g_val = parse_coords(first_valid["task"], "gripper")
            s_val = parse_coords(first_valid["task"], "sponge")
            for e in entries:
                if "?" in e["task"]:
                    new_task = "Pick up a yellow sponge and place it inside the box"
                    new_task = replace_coords(new_task, "gripper", g_val)
                    new_task = replace_coords(new_task, "sponge", s_val)
                    e["task"] = new_task
                    modified += 1
                else:
                    break

        # 再处理结尾缺失
        last_valid = next((e for e in reversed(entries) if "?" not in e["task"]), None)
        if last_valid:
            g_val = parse_coords(last_valid["task"], "gripper")
            s_val = parse_coords(last_valid["task"], "sponge")
            for e in reversed(entries):
                if "?" in e["task"]:
                    new_task = "Pick up a yellow sponge and place it inside the box"
                    new_task = replace_coords(new_task, "gripper", g_val)
                    new_task = replace_coords(new_task, "sponge", s_val)
                    e["task"] = new_task
                    modified += 1
                else:
                    break

        # 再处理中间插值
        for i, entry in enumerate(entries):
            task = entry["task"]
            if "?" in task:
                j = i - 1
                while j >= 0 and "?" in entries[j]["task"]:
                    j -= 1
                k = i + 1
                while k < len(entries) and "?" in entries[k]["task"]:
                    k += 1

                if j >= 0 and k < len(entries):
                    g1 = parse_coords(entries[j]["task"], "gripper")
                    g2 = parse_coords(entries[k]["task"], "gripper")
                    s1 = parse_coords(entries[j]["task"], "sponge")
                    s2 = parse_coords(entries[k]["task"], "sponge")

                    g_interp = linear_interpolate(g1, g2, k - j)
                    s_interp = linear_interpolate(s1, s2, k - j)

                    for m in range(j + 1, k):
                        if "?" in entries[m]["task"]:
                            new_task = "Pick up a yellow sponge and place it inside the box"
                            new_task = replace_coords(new_task, "gripper", g_interp[m - j - 1])
                            new_task = replace_coords(new_task, "sponge", s_interp[m - j - 1])
                            entries[m]["task"] = new_task
                            modified += 1
                else:
                    unmodified += 1
            else:
                if "sachet" in task:
                    entries[i]["task"] = task.replace("sachet", "sponge")

    with open(output_file, "w") as f:
        for ep in sorted(episodes):
            for entry in sorted(episodes[ep], key=lambda x: x["frame_index"]):
                f.write(json.dumps(entry, ensure_ascii=False) + "\n")

    print(f"修改成功: {modified}, 未修改成功: {unmodified}")


if __name__ == "__main__":
    input_file = "/home/zhonglinyue/code/lerobot_diy/training_dataset/831/sponge_pickup/meta/modified_tasks_pure.jsonl"
    output_file = "full_modified_tasks.jsonl"
    process_file(input_file, output_file)


修改成功: 356, 未修改成功: 0


变成mtask_relative

In [8]:
import json
import re
from pathlib import Path


def parse_coords(text, key):
    pattern = rf"{key} at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)"
    match = re.search(pattern, text)
    if match:
        return tuple(map(float, match.groups()))
    return None


def process_file(input_file, output_file):
    lines = [json.loads(line) for line in Path(input_file).read_text().splitlines()]

    with open(output_file, "w") as f:
        for entry in lines:
            task = entry["task"]
            g = parse_coords(task, "gripper")
            s = parse_coords(task, "sponge")

            if g and s:
                rel = tuple(s_i - g_i for s_i, g_i in zip(s, g))
                new_task = (
                    f"Pick up a yellow sponge and place it inside the box, "
                    f"sponge position relative to gripper is ({rel[0]:.3f}, {rel[1]:.3f}, {rel[2]:.3f})"
                )
                entry["task"] = new_task

            f.write(json.dumps(entry, ensure_ascii=False) + "\n")


if __name__ == "__main__":
    input_file = "/home/zhonglinyue/code/lerobot_diy/training_dataset/831/sponge_pickup/meta/full_modified_tasks.jsonl"
    output_file = "/home/zhonglinyue/code/lerobot_diy/training_dataset/831/sponge_pickup/meta/lmtask_relative.jsonl"
    process_file(input_file, output_file)


5cm格子

In [9]:
import json
import re
from pathlib import Path


def parse_coords(text, key):
    pattern = rf"{key} at \((-?\d+\.?\d*), (-?\d+\.?\d*), (-?\d+\.?\d*)\)"
    match = re.search(pattern, text)
    if match:
        return tuple(map(float, match.groups()))
    return None


def process_file(input_file, output_file):
    lines = [json.loads(line) for line in Path(input_file).read_text().splitlines()]

    with open(output_file, "w") as f:
        for entry in lines:
            task = entry["task"]
            g = parse_coords(task, "gripper")
            s = parse_coords(task, "sponge")

            if g and s:
                rel = tuple(s_i - g_i for s_i, g_i in zip(s, g))
                # 转换到 5cm 格子并取整
                grid = tuple(round(val / 0.05) for val in rel)
                new_task = (
                    f"Pick up the pyramid-shaped sponge and place it into the box, "
                    f"sponge position relative to gripper is ({grid[0]}, {grid[1]}, {grid[2]}) in 5cm grid units."
                )
                entry["task"] = new_task

            f.write(json.dumps(entry, ensure_ascii=False) + "\n")


if __name__ == "__main__":
    input_file = "/home/zhonglinyue/code/lerobot_diy/training_dataset/831/sponge_pickup/meta/full_modified_tasks.jsonl"
    output_file = "/home/zhonglinyue/code/lerobot_diy/training_dataset/831/sponge_pickup/meta/mtask_relative_5cm.jsonl"
    process_file(input_file, output_file)
